# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [33]:
%matplotlib widget

In [34]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [35]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract image data 

## Read dataframe

In [36]:
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/',
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [37]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200901_062829_bsize10_512-1proc', '20200825_100837…

In [38]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200831_164511_bsize10_512


In [14]:
# main_dir=''

In [39]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

In [40]:
print(df.shape)
df.head()


(32998, 19)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
0,0,0,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,156.409874,0.568250,0.013389,156.991512,11.802867,140.521809,131782.763292,6.417095e+07,0,0,"[4.656314861863851e-05, 0.04946572788228852, 0...","[1.2239528435903247e-05, 0.003698883829943908,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15816246049.41213, 59096.80031838417, 28983.4...","[35558592.02687507, 3306.216730529061, 1420.67..."
1,0,1,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,157.810828,0.757574,0.013389,158.581790,11.714393,142.961043,133939.347834,8.160899e+07,0,0,"[5.482435240581631e-05, 0.05818837890051619, 0...","[1.4211921854194027e-05, 0.003934476788837656,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15761435250.079742, 65965.76474971771, 29000....","[36285835.229485944, 3696.5637490518106, 1353...."
2,0,2,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,155.492982,0.980313,0.013389,156.486684,11.493092,66.006846,129221.638737,1.364079e+08,0,0,"[8.711814902842042e-05, 0.060228896233829085, ...","[1.3274132944042214e-05, 0.0017634969758215202...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15793555872.692316, 71631.31405143738, 35088....","[15597858.22439727, 5896.06376268714, 1870.782..."
3,0,3,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,150.989292,1.425727,0.013389,152.428409,11.142338,57.495163,166987.782866,1.872069e+08,0,0,"[0.0001366853717514872, 0.06755894432937934, 0...","[1.5457121689665846e-05, 0.001735133964755206,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[15907978133.064386, 50154.989252471925, 25123...","[11761484.52690822, 2669.564516256137, 587.952..."
4,0,4,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,139.749615,2.703870,0.013389,142.466875,10.499410,43.605505,259533.497890,2.460557e+08,0,0,"[0.0001727342610046267, 0.0756785274717696, 0....","[2.108209466173035e-05, 0.0014744493022307251,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[16175057397.244043, 32265.785321998595, 17070...","[6768570.687627409, 2866.9264646460074, 776.25..."


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err


### View best epochs

#### Locations with best chi_sqr

In [55]:
chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]

,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
31743,14,31743,0.000115,0.007121,0.103561,0.110796,0.000083,17.376732,0.404873,7.224817e+02
15028,6,15028,0.042877,0.000495,12.104539,12.147911,0.041814,24.838981,1.847823,1.253451e+03
6936,3,6936,0.490089,0.081638,0.001139,0.572865,0.292050,30.326860,5.204973,1.427339e+04
28206,12,28206,0.000861,0.003000,0.003611,0.007472,0.000349,36.567403,1.633367,1.096173e+03
32420,14,32420,0.000257,0.001333,0.099063,0.100653,0.000036,30.487295,0.195482,3.284864e+02
3040,1,3040,1.201529,0.291189,0.013389,1.506107,0.668475,2.623033,2965.083517,9.036817e+06
30332,13,30332,0.010372,0.004587,0.022456,0.037415,0.006085,24.676371,0.059244,2.511601e+02
22232,9,22232,0.002668,0.003948,0.684377,0.690994,0.001892,44.495939,0.103958,4.506815e+01


In [42]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

In [43]:
df_sliced=f_get_best_chisqr_models(df,cutoff=0.3)
print(df_sliced.shape)

(4099, 19)


In [44]:
df_sliced.sort_values(by=['chi_1','chi_spec1'])[['epoch','step','chi_1','chi_spec1']].head(5)

,epoch,step,chi_1,chi_spec1
32942,14,32942,0.007982,0.174309
31341,13,31341,0.008565,0.147168
23777,10,23777,0.008699,0.430917
32886,14,32886,0.008710,0.303737
32562,14,32562,0.009277,0.247334


In [46]:
### Plot chi-sqr values
df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### High Pixel images

In [47]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

In [48]:
df[df.num_vlarge>10].tail()

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err


## Compare samples

In [61]:
# df_sliced.sort_values(by=['chi_spec1','chi_1']).head(5)
# df_sliced.sort_values(by=['chi_1','chi_spec1']).head(5)

In [56]:
best_idx=[]
best_idx.append(df_sliced.sort_values(by=['chi_1','chi_spec1']).head(2).index)
best_idx.append(df_sliced.sort_values(by=['chi_spec1','chi_1']).head(2).index)

best_idx=[i for j in best_idx for i in j]
print(best_idx)

df_best=df_sliced.loc[best_idx]
print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])
df_best

[32942, 31341, 30332, 30091]
[(14, 32942), (13, 31341), (13, 30332), (13, 30091)]


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
32942,14,32942,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.004089,0.002167,0.001726,0.007982,0.003349,22.046198,0.174309,411.480653,0,0,"[1.0855972794877735, 1.0048586026411805, 0.265...","[0.001671947730229744, 0.002414049449283854, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[50253728906.279686, 2673503.5788085936, 23853...","[60263749.71345505, 375038.82666581497, 166590..."
31341,13,31341,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.000642,0.003218,0.004706,0.008565,0.000336,26.724849,0.147168,307.577972,0,0,"[1.1303887754033255, 0.9464471712386422, 0.257...","[0.006089816302573856, 0.007004901954703871, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[50829036846.51514, 2580027.092529297, 2005139...","[73303387.34740868, 216967.88160488041, 135753..."
30332,13,30332,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.010372,0.004587,0.022456,0.037415,0.006085,24.676371,0.059244,251.160112,1,0,"[1.0688762607821372, 1.0128357699787984, 0.277...","[0.004662908569698298, 0.0024210509268234367, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[49980999959.57217, 2748650.114306641, 2262697...","[80404175.17669658, 152678.86045233998, 190166..."
30091,13,30091,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.015094,0.006611,0.076373,0.098077,0.007479,55.150103,0.071572,59.603050,1,0,"[1.0618883756370994, 1.0097350732144883, 0.291...","[0.01830814852673116, 0.014093433590341138, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[49875951955.06299, 2393372.605126953, 2075707...","[289584741.556528, 215253.80035150034, 164139...."


## Plot pixel intensity and spectrum

In [50]:
def f_plot_hist_spec_combined(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label='{0}_{1}_{2}'.format(i,row.epoch,row.step)
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   

    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)

    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    
    


In [58]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[:4000][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)


(4000, 512, 512)


In [59]:
f_plot_hist_spec_combined(df_best,dict_val)
# f_plot_hist_spec_combined(df[df.epoch==10].head(10),dict_val)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## View image block

In [60]:
def f_get_img(df,epoch,step):
    df_temp=df[(df.epoch==epoch)&(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,13,30091)

In [61]:
f_plot_grid(img_arr[:18],cols=6,fig_size=(10,6))

2 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,6))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
df_sliced[df_sliced.epoch==4]['step'].values

In [ ]:
df[(df.epoch==18)&(df.step>=15975)&(df.step<=15983)][['epoch','step','chi_1','chi_spec1','num_vlarge']]

In [30]:
df[df.epoch==10].head(10).shape

(10, 19)